In [1]:
from utils import *
import numpy as np
from tqdm import tqdm
from copy import deepcopy
import heapq

# Parsing

In [2]:
def get_grid(inlines):
    mat = np.array([[float(c) for c in l] for l in inlines.split("\n")])
    return mat
alldirs = [(0,1),(1,1),(1,0),(-1,0),(-1,-1),(0,-1),(-1,1),(1,-1)]
dirs = [(0,1),(1,0),(-1,0),(0,-1)]

def parse(inlines):
    return get_grid(inlines)

ls = parse(open("data/input10.txt").read())

# Part 1

In [3]:
class Node:
    def __init__(self, cost, ind):
        self.ind = ind
        self.cost = cost
    def __lt__(self, other):
        return self.cost < other.cost

def step(dims, grid, curpos):
    newpos = []
    for d in dirs:
        if 0<=curpos[0]+d[0]<dims[0] and 0<=curpos[1]+d[1]<dims[1] and grid[curpos[0]+d[0],curpos[1]+d[1]]==grid[curpos[0],curpos[1]]+1:
            newpos.append((curpos[0]+d[0], curpos[1]+d[1]))
    return newpos

def dijkstra(grid, startpos, goalpos):
    state2ind = {}
    state2ind[startpos] = 0
    nnodes = 1
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if (i,j)!=startpos:
                state2ind[(i,j)] = nnodes
                nnodes += 1
    ind2state = {v:k for k,v in state2ind.items()}

    q = []
    curstate = startpos
    heapq.heappush(q, Node(0, state2ind[curstate]))
    amap = {state2ind[curstate]: 0}
    visited = [False]*nnodes
    visited[state2ind[curstate]] = True

    for i in range(nnodes):
        if not q:
            break
        curnode = heapq.heappop(q)
        while visited[curnode.ind]:
            if not q:
                break
            curnode = heapq.heappop(q)
        curind = curnode.ind
        curcost = curnode.cost
        curstate = ind2state[curind]
        visited[state2ind[curstate]] = True
        for newstate in step(grid.shape, grid, curstate):
            newind = state2ind[newstate]
            if visited[newind]==False:
                newcost = curcost + 1
                if newind not in amap:
                    amap[newind] = newcost
                else:
                    if newcost<amap[newind]:
                        amap[newind] = newcost
                heapq.heappush(q, Node(newcost, newind))
    
    mincost = float('inf')
    for nodeind, nodecost in amap.items():
        if ind2state[nodeind]==goalpos:
            if nodecost<mincost:
                mincost = nodecost
    return mincost

stpos = np.where(ls==0)
epos = np.where(ls==9)
accum = 0
for x1,y1 in tqdm(zip(stpos[0],stpos[1])):
    acc = 0
    for x2,y2 in zip(epos[0],epos[1]):
        mincost = dijkstra(ls, (x1,y1), (x2,y2))
        if mincost<float('inf'):
            acc += 1
    accum += acc
accum

259it [00:42,  6.05it/s]


629

# Part 2

In [4]:
class Node:
    def __init__(self, cost, ind):
        self.ind = ind
        self.cost = cost
    def __lt__(self, other):
        return self.cost < other.cost

def step(dims, grid, curpos, occ):
    newpos = []
    for d in dirs:
        if 0<=curpos[0]+d[0]<dims[0] and 0<=curpos[1]+d[1]<dims[1] and grid[curpos[0]+d[0],curpos[1]+d[1]]==grid[curpos[0],curpos[1]]+1:# and occ[curpos[0]+d[0],curpos[1]+d[1]]!=1:
            newpos.append((curpos[0]+d[0], curpos[1]+d[1]))
    return newpos


def findnpaths(grid, cpos, accum, occ):
    occcop = np.copy(occ)
    occcop[cpos] = 1
    newpos = step(grid.shape, grid, cpos, occcop)
    for p in newpos:
        if grid[p]==9:
            accum += [1]
        else:
            findnpaths(grid, p, accum, occcop)

stpos = np.where(ls==0)
accum = 0
for x1,y1 in tqdm(zip(stpos[0],stpos[1])):
    npaths = []
    occ = np.zeros(ls.shape)
    occ[y1,x1] = 1
    findnpaths(ls, (x1,y1), npaths, occ)
    accum += np.sum(npaths)
int(round(accum))

259it [00:00, 5081.18it/s]


1242